## clinical variable retrieval codebase.
each querey is referencing a sql querey linked in my github for ALL patients in the database, then generating a dataframe, then paring that dataframe down to only the patients/icustay_id in our cohort. 

In [3]:
#7-15-18
#the final version of this notebook seeks to accomplish a few tasks:
#organize and standardize all sql code so that i am running a .sql file rather than typing sql code into jupyter
#change all filepaths to match the github linked directory to ensure better version control
#extract all of the structured clinical variables we need for our project

time: 889 µs


In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

In [2]:
from pathlib import Path
os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


time: 1.67 ms


In [3]:
#patients of interest from rotation_cohort_generation

final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/25012019_final_pt_PA_df2.csv', index_col=0) #final cohort for PA 
date='25012019'

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 62.8 ms


## Access MIMIC database and convert it to dataframe in Pandas

In [43]:
import psycopg2
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

Password: ·····


time: 1.94 s


In [5]:
#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_exe_show(sql_sentence):
    cur.execute(sql_sentence)
    rows = cur.fetchall()
    col = []
    for i in range(len(cur.description)):
        col.append(cur.description[i][0])
    table = pd.DataFrame(rows,columns=col)
    return table

def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['icustay_id'].isin(icustay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df

def sql_to_df_hadm(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['hadm_id'].isin(hadm_id)]
    return sql_exe_show_df

def clinvar_fxn(var_name, path):
    f= open(path, 'r')
    var = f.read()
    cur.execute('rollback')
    cur.execute(var)
    try:
        df= sql_to_df_icu(sql_exe_show('select * from %s;' %(var_name)))
    except KeyError or NameError:
        try:      
            df= sql_to_df_hadm(sql_exe_show('select * from %s;' %(var_name)))
        except KeyError or NameError:
            df= sql_to_df_patients(sql_exe_show('select * from %s;' %(var_name)))

    print(df.shape)
    return(df)


time: 47.8 ms


# extracting clinical data for our patients

## Echodata in Noteevents

In [6]:
echodata_df= clinvar_fxn(
    'echodata',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/echodata.sql' 
)

(7450, 18)
time: 34.8 s


## Weight

In [7]:
# -- This query extracts weights for adult ICU patients on their first ICU day.
# -- It does *not* use any information after the first ICU day, as weight is
# -- sometimes used to monitor fluid balance.

weightfirstday_df= clinvar_fxn(
    'weightfirstday',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/weightfirstday.sql' 
)
weightfirstday_df['uom']='kg'

(8731, 6)
time: 2.01 s


## Height

In [8]:
# -- This query extracts heights for adult ICU patients.
# -- It uses all information from the patient's first ICU day.
# -- This is done for consistency with other queries - it's not necessarily needed.
# -- Height is unlikely to change throughout a patient's stay.
heightfirstday_df= clinvar_fxn(
    'heightfirstday',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/heightfirstday.sql' 
)

heightfirstday_df['uom']='cm'

(8731, 4)
time: 1.31 s


# Labs

In [9]:
# -- This query pivots lab values for all patients, then filtered to those in my cohort.
# -- Have already confirmed that the unit of measurement is always the same: null or the correct unit

labs_all_nosummary_df= clinvar_fxn(
    'labs_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/labs_all_nosummary.sql' 
)


(2231976, 6)
time: 4min 54s


In [10]:
#importing unit of mesurements:
def uom_sql_import(file_path):
    if isinstance(file_path, str)== True:
        f = open(Path(file_path), 'r')
    else:
        f = open(Path(str(file_path)), 'r')

    SQL = open(file_path,'r').read()
    SQL_df= pd.read_sql_query(SQL,conn)   
    return(SQL_df)

time: 8.13 ms


In [11]:
lab_uom= uom_sql_import(Path(wd+'/data/raw/clinical_var_sql/unit_of_mesurement/labs_uom.sql'))
labs_all_nosummary_df = pd.merge(labs_all_nosummary_df, lab_uom, left_on='label', right_on='label')

time: 22.5 s


In [12]:
wd

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'

time: 5.45 ms


## Vital Sign

In [13]:
##-- This query pivots the vital signs for the first 24 hours of a patient's stay
##-- Vital signs include heart rate, blood pressure, respiration rate, and temperature

vitals_all_nosummary_df= clinvar_fxn(
    'vitals_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/vitals_all_nosummary.sql' 
)

(10389099, 7)
time: 23min 12s


In [14]:
vitals_all_nosummary_df.head()

,subject_id,hadm_id,icustay_id,charttime,valueuom,vitalid,valuenum
3,9,150750,220597,2149-11-11 00:00:00,mmHg,MeanArtPress,81.0
4,9,150750,220597,2149-11-11 19:00:00,mmHg,MeanArtPress,72.0
5,9,150750,220597,2149-11-11 19:45:00,mmHg,MeanArtPress,76.0
6,9,150750,220597,2149-11-11 20:00:00,mmHg,MeanArtPress,99.0
17,25,129635,203487,2160-11-02 15:00:00,mmHg,MeanArtPress,82.0


time: 1.78 s


In [15]:
vitals_all_nosummary_df.head()

,subject_id,hadm_id,icustay_id,charttime,valueuom,vitalid,valuenum
3,9,150750,220597,2149-11-11 00:00:00,mmHg,MeanArtPress,81.0
4,9,150750,220597,2149-11-11 19:00:00,mmHg,MeanArtPress,72.0
5,9,150750,220597,2149-11-11 19:45:00,mmHg,MeanArtPress,76.0
6,9,150750,220597,2149-11-11 20:00:00,mmHg,MeanArtPress,99.0
17,25,129635,203487,2160-11-02 15:00:00,mmHg,MeanArtPress,82.0


time: 13.7 ms


In [16]:
# vitals_all_nosummary_df = vitals_all_nosummary_df.rename(index=str, columns={"valueuom":"uom"}) #change valueom to uom
# vitals_all_nosummary_nonull = vitals_all_nosummary_df.loc[vitals_all_nosummary_df['vitalid'].notnull(),:]#.count() #6930 NULL values

# vitals_all_nosummary_nonull.loc[vitals_all_nosummary_nonull.loc[:,'uom']=='BPM','uom']='bpm'

# #overall the values are extremely similar and are likely the same thing
# #i will combine them.
# vitals_all_nosummary_nonull.loc[
#     (vitals_all_nosummary_nonull['vitalid']=='RespRate') & 
#     (vitals_all_nosummary_nonull['uom']=='bpm'),'uom']='insp/min'

# vitals_all_nosummary_nonull.loc[
#     (vitals_all_nosummary_nonull['vitalid']=='TempC') & 
#     (vitals_all_nosummary_nonull['uom']=='?C'),'uom']='Deg. C'

# vitals_all_nosummary_nonull.loc[
#     (vitals_all_nosummary_nonull['vitalid']=='TempF') & 
#     (vitals_all_nosummary_nonull['uom']=='Deg. F'),'uom']='Deg. C'

# vitals_all_nosummary_nonull.loc[
#     (vitals_all_nosummary_nonull['vitalid']=='TempF') & 
#     (vitals_all_nosummary_nonull['uom']=='?F'),'uom']='Deg. C'


# vitals_all_nosummary_nonull.loc[
#     (vitals_all_nosummary_nonull['vitalid']=='TempF'),'vitalid']='TempC'

time: 12 ms


## Glasgow Coma Scale

In [17]:
gcsall_df= clinvar_fxn(
    'gcsall',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/gcsall.sql' 
)

(8731, 8)
time: 1min 38s


In [18]:
#modified on 8/28/18 to have the days annotation.
##--8/28/18: added in epoch as days, in order to help determine btwn t_0 and 72 hour for pts.
gcsall_days_df= clinvar_fxn(
    'gcsall_days',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/gcsall_days.sql' 
)



(8731, 9)
time: 56.3 s


In [19]:
#need to add in admit time to pts
def gcs_72(gcsall_days_df,final_pt_df2):
    gcsall_days_df_merge= pd.merge(
    gcsall_days_df,
    final_pt_df2[['icustay_id','ICU_admit','t_0']],
    left_on='icustay_id',
    right_on='icustay_id')

    gcsall_days_df_merge['admit_plus_day']= (
        pd.to_datetime(gcsall_days_df_merge['ICU_admit'])
        + pd.to_timedelta(gcsall_days_df_merge['day'], unit='D')
    )

    gcsall_days_df_merge['delta']= (
        pd.to_datetime(gcsall_days_df_merge['admit_plus_day']) - pd.to_datetime(gcsall_days_df_merge['t_0'])
    )
    gcsall_days_df_merge_72= (
        gcsall_days_df_merge.loc[gcsall_days_df_merge.loc[:,'delta']>= pd.Timedelta(days=0),:])
    gcsall_days_df_merge_72= (
        gcsall_days_df_merge_72.loc[gcsall_days_df_merge_72.loc[:,'delta']<= pd.Timedelta(days=3),:])
    return(gcsall_days_df_merge_72)

time: 38.3 ms


In [20]:
gcs72_df = gcs_72(gcsall_days_df,final_pt_df2)

gcs72_df['uom']='GCS_score' #adding in uom

time: 753 ms


In [21]:
# date= '27082018'

# pd.DataFrame(gcs72_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcs72_df.csv' %(date))

time: 711 µs


## Renal replacement therapy

In [22]:
# rrt_all_df= clinvar_fxn(
#     'rrt_all',
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/rrt_all.sql' 
# )

time: 677 µs


the sql code for this was not equipped to join all of the charttimes together. so i decided to do it in python below.
the rrt_all_df code above was only a 1 or 0 if patient had RRT during their entire icu stay. 

- step 1: run all sql codes
- 2: filter on only the t_0 to t_72 hour rows
- 3: filter on the 1223 patients who have a positive value
- 4: get the earliest incidence of rrt for each 1223 patients. 

In [23]:
#mv_ce
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_ce.sql', 'r')
rrtSQL_mv_ce = f.read()
rrtSQL_mv_ce_sql = query_schema + rrtSQL_mv_ce.format(tuple(patients))
rrtSQL_mv_ce_df=pd.read_sql_query(rrtSQL_mv_ce_sql,conn)            

time: 16.3 s


In [24]:
#cv
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_cv.sql', 'r')
rrtSQL_cv = f.read()
rrtSQL_cv_sql = query_schema + rrtSQL_cv.format(tuple(patients))
rrtSQL_cv_df=pd.read_sql_query(rrtSQL_cv_sql,conn)          

time: 26.2 s


In [25]:
#mv_ie
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_ie.sql', 'r')
rrtSQL_mv_ie = f.read()
rrtSQL_mv_ie_sql = query_schema + rrtSQL_mv_ie.format(tuple(patients))
rrtSQL_mv_ie_df=pd.read_sql_query(rrtSQL_mv_ie_sql,conn)      
rrtSQL_mv_ie_df['charttime']= rrtSQL_mv_ie_df['starttime']
rrtSQL_mv_ie_df=rrtSQL_mv_ie_df.drop('starttime', axis=1)

time: 3.15 s


In [26]:
#mv_de
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_de.sql', 'r')
rrtSQL_mv_de = f.read()
rrtSQL_mv_de_sql = query_schema + rrtSQL_mv_de.format(tuple(patients))
rrtSQL_mv_de_df=pd.read_sql_query(rrtSQL_mv_de_sql,conn)          

time: 2.03 s


In [27]:
#mv_pe
f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_pe.sql', 'r')
rrtSQL_mv_pe = f.read()
rrtSQL_mv_pe_sql = query_schema + rrtSQL_mv_pe.format(tuple(patients))
rrtSQL_mv_pe_df=pd.read_sql_query(rrtSQL_mv_pe_sql,conn)          
rrtSQL_mv_pe_df['charttime']= rrtSQL_mv_pe_df['starttime']
rrtSQL_mv_pe_df=rrtSQL_mv_pe_df.drop('starttime', axis=1)

time: 303 ms


In [28]:
def hour_72_window_rrt(df, final_pt_df2,timevar):

    df= pd.merge(final_pt_df2[['icustay_id','t_0']], df, left_on= 'icustay_id', right_on = 'icustay_id', how='left') #n=240317
    df['delta']= pd.to_datetime(df[timevar]) - pd.to_datetime(df['t_0'])
    df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=0),:]
    df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=3),:] 
    #df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'rrt']==1,:].groupby('icustay_id')['charttime'].min()

    return(pd.DataFrame(df_after_t0))#.reset_index())

time: 6.88 ms


In [29]:
rrtSQL_mv_ce_pt =hour_72_window_rrt(rrtSQL_mv_ce_df, final_pt_df2, 'charttime')
rrtSQL_cv_pt =hour_72_window_rrt(rrtSQL_cv_df, final_pt_df2, 'charttime')
rrtSQL_mv_ie_pt =hour_72_window_rrt(rrtSQL_mv_ie_df, final_pt_df2, 'charttime')
rrtSQL_mv_de_pt =hour_72_window_rrt(rrtSQL_mv_de_df, final_pt_df2, 'charttime')
rrtSQL_mv_pe_pt =hour_72_window_rrt(rrtSQL_mv_pe_df, final_pt_df2, 'charttime')

time: 614 ms


In [31]:
rrt_merged_pt= pd.concat([rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt])
rrt_merged_pt= pd.DataFrame(rrt_merged.loc[rrt_merged.loc[:,'rrt']==1,:].groupby('icustay_id')['charttime'].min().reset_index())
rrt_merged_pt['rrt']=1

rrt_merged_allpt_df= pd.merge(final_pt_df2[['icustay_id','t_0']], rrt_merged_pt, left_on= 'icustay_id', right_on = 'icustay_id', how='left') #n=240317
rrt_merged_allpt_df=rrt_merged_allpt_df.rename(index=str, columns={"charttime":"first_charttime"})

rrt_merged_allpt_df['uom']='category' #adding a uom category


###RRTMERGED NOT WORKING

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


NameError: name 'rrt_merged' is not defined

time: 27.5 ms


In [ ]:
rrt_merged_allpt_df_notnull = rrt_merged_allpt_df.loc[rrt_merged_allpt_df['rrt'].notnull(),:]

In [ ]:
#rrt_merged_allpt_df #n=1223 patients with positive rrt. charttime listed in this sheet is the first instance in t_0:t_72

In [ ]:
# date= '27082018'

# pd.DataFrame(rrt_merged_allpt_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_rrt_merged_allpt_df.csv' %(date))

## Urine Output

In [34]:
urine_output_all_df= clinvar_fxn(
    'urine_output_all',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/urine_output_all.sql' 
)

(53181, 5)
time: 11.2 s


# UTI related variables

In [35]:
uti_all_df= clinvar_fxn(
    'uti_all',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/uti_all.sql' 
)

(501177, 12)
time: 2min 25s


## Blood Gas Test

In [36]:
def PaO2(bg_all_nosummary_df):
    """
    overview: replaces the PO2 label with PaO2 on all instances (defined as sharing icustay_id and charttime being equal)
        where the specimen label == 'ART'
        
    input: bloodgas dataframe with values annotated. 
    output: bloodgas dataframe with values annotated where PO2 label is replaced with PaO2 according to above criteria
    """
    
    #making a unique varaible to search for and mark all rows where time and icustay_id has an art flag
    bg_all_nosummary_df['unique_var']= bg_all_nosummary_df['icustay_id'].map(str) +  bg_all_nosummary_df['charttime'].map(str)

    #making subset dataframe for label == SPECIMEN 
    bg_all_nosummary_specimen= bg_all_nosummary_df.loc[bg_all_nosummary_df.loc[:,'label']=='SPECIMEN',:]
    
    #all ART related rows: unique_var for all rows where label== SPECIMEN
    bg_all_nosummary_ART = bg_all_nosummary_specimen[bg_all_nosummary_specimen['value']=='ART'] 
    bg_all_nosummary_ART_list= list(bg_all_nosummary_ART['unique_var'].unique())

    #two criteria needed to change the PO2 to PaO2 label.
    criteria1=(bg_all_nosummary_df['label'] == 'PO2')
    criteria2=(bg_all_nosummary_df['unique_var'].isin(bg_all_nosummary_ART_list))
    
    #making changes
    bg_all_nosummary_df.loc[(criteria2 & criteria1),'label']= 'PaO2'
    
    return(bg_all_nosummary_df)

time: 22.5 ms


In [37]:
bg_all_nosummary_df= clinvar_fxn(
    'bg_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/bg_all_nosummary.sql' 
) 
bg_all_nosummary_df = PaO2(bg_all_nosummary_df)


(1213548, 8)
time: 2min 27s


In [38]:
bg_all_nosummary_df.head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,valueuom,unique_var
607,9,150750,220597,2149-11-09 17:47:00,BASEEXCESS,1.0,1,mEq/L,2205972149-11-09 17:47:00
608,9,150750,220597,2149-11-09 17:47:00,CHLORIDE,103.0,103,mEq/L,2205972149-11-09 17:47:00
609,9,150750,220597,2149-11-09 17:47:00,FIO2,60.0,60,None,2205972149-11-09 17:47:00
610,9,150750,220597,2149-11-09 17:47:00,GLUCOSE,178.0,178,mg/dL,2205972149-11-09 17:47:00
611,9,150750,220597,2149-11-09 17:47:00,HEMATOCRIT,42.0,42,%,2205972149-11-09 17:47:00


time: 13.2 ms


## Vaso_active therapies

In [42]:
# 10/12/18 added amountuom as amount_uom, rateuom as rate_uom to many lines of the sql code.

###WEIGHT DURATION NOT WORKING
weightdurations_df= clinvar_fxn(
    'weightdurations',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/weightdurations.sql'
)
#
epi_dose_df= clinvar_fxn(
    'epinephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/epinephrine_dose.sql'
)
#
norepi_dose_df= clinvar_fxn(
    'norepinephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/norepinephrine_dose.sql'
)
#
dopamine_dose_df= clinvar_fxn(
    'dopamine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/dopamine_dose.sql'
)
#
dobutamine_dose_df= clinvar_fxn(
    'dobutamine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/dobutamine_dose.sql'
)
#
vasopressin_dose_df= clinvar_fxn(
    'vasopressin_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/vasopressin_dose.sql'
)

#removing units/hour, as these are not appropriate
vasopressin_dose_df= vasopressin_dose_df.loc[~vasopressin_dose_df.loc[:,'rate_uom'].isin(['Uhr','units/hour']),:]
#
phenylephrine_dose_df= clinvar_fxn(
    'phenylephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/phenylephrine_dose.sql'
)

ProgrammingError: relation "weightdurations" does not exist
LINE 34:   left join weightdurations wd
                     ^


time: 49.2 ms


### uom testing

In [ ]:
dopamine_dose_df.head()

In [ ]:
dopamine_dose_df['rate_uom'].value_counts()

In [ ]:
norepi_dose_df.head()

In [ ]:
norepi_dose_df['rate_uom'].value_counts()

In [ ]:
dobutamine_dose_df.head()

In [ ]:
dobutamine_dose_df['rate_uom'].value_counts()

In [ ]:
vasopressin_dose_df.head()

In [ ]:
vasopressin_dose_df['rate_uom'].value_counts()

In [ ]:
# #removing units/hour, as these are not appropriate
# vasopressin_dose_df= vasopressin_dose_df.loc[~vasopressin_dose_df.loc[:,'rate_uom'].isin(['Uhr','units/hour']),:]

In [ ]:
phenylephrine_dose_df['rate_uom'].value_counts()

In [ ]:
#dopamine_dose_df['amount_uom'].value_counts()
dopamine_dose_df['rate_uom'].value_counts()

In [ ]:
epi_dose_df.head()

In [ ]:
epi_dose_df['rate_uom'].value_counts()

In [ ]:
#adding an identification label column and merging them into 1 df. 
epi_dose_df['label']='epinephrine'
norepi_dose_df['label']='norepinephrine'
dopamine_dose_df['label']='dopamine'
dobutamine_dose_df['label']='dobutamine'
vasopressin_dose_df['label']='vasopressin'
phenylephrine_dose_df['label']='phenylephrine'
vaso_dose_df = pd.concat([epi_dose_df, norepi_dose_df, dopamine_dose_df, dobutamine_dose_df, vasopressin_dose_df,phenylephrine_dose_df ])

#removing u/hour

In [ ]:
vaso_dose_df.head()

# ventilator settings and categorization

In [ ]:
#ventsettings_df = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/15082018_ventsettings_df.csv', index_col=0)
ventsettings_df= clinvar_fxn(
    'ventsettings',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/ventsettings.sql'
)
#going from wide format to long:
#pd.melt(ventsettings_df, id_vars=['icustay_id','charttime'])

In [ ]:
def vent_categorization(final_pt_df2, ventsettings_df, time_var='t_0'):
    """
    overview: provides 3 variables, one for each 24 hour peroid following first time_var (default is t_0 ), 
    that codes vent status during that 24 hr peroid.
    ventilation data is based upon the ventsettings.sql file output. 
       categories: 
        - Vent- had mech vent within 24hr period
        - Oxygen- had only oxygentherapy within 24hr period
        - None- had neither within 24hr period.
        - Not_coded= data was not present
        
    input: final cohort dataframe (by icustay_id/hadm_id) annotated with t_0
    output: a dataframe with every icustay_id as a row, and a categorization for each of the first 3 days based on ventilation recieved:
    """
    
    ventsettings_merge_df= pd.merge(ventsettings_df,
                                final_pt_df2[['icustay_id',time_var]],
                                left_on='icustay_id',
                                right_on= 'icustay_id',
                                how='left')
 
    ventsettings_df_3dayannotation= final_pt_df2[['icustay_id',time_var]]
    
    #origionally coded as: Not_coded as the default. changed to none at Nelson's request
    ventsettings_df_3dayannotation['vent_day1']='None' #coding Not_coded as the default. must be sure to code over this to None to ensure I capture everyone
    ventsettings_df_3dayannotation['vent_day2']='None' #coding Not_coded as the default. must be sure to code over this to None to ensure I capture everyone
    ventsettings_df_3dayannotation['vent_day3']='None' #coding Not_coded as the default. must be sure to code over this to None to ensure I capture everyone
    #ventsettings_df_3dayannotation.head()

    day1= (
        (pd.to_datetime(ventsettings_merge_df[time_var])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('00:00:00','23:59:59'))
    day2= (
        (pd.to_datetime(ventsettings_merge_df[time_var])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('24:00:00','47:59:59'))
    day3= (
        (pd.to_datetime(ventsettings_merge_df[time_var])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('48:00:00','71:59:59'))

    mech_1= list(ventsettings_merge_df.loc[day1 & ventsettings_merge_df['mechvent']==1 ,'icustay_id'])
    mech_2= list(ventsettings_merge_df.loc[day2 & ventsettings_merge_df['mechvent']==1 ,'icustay_id'])
    mech_3= list(ventsettings_merge_df.loc[day3 & ventsettings_merge_df['mechvent']==1 ,'icustay_id'])
    O2_1= list(ventsettings_merge_df.loc[day1 & ventsettings_merge_df['oxygentherapy']==1 ,'icustay_id'])
    O2_2= list(ventsettings_merge_df.loc[day2 & ventsettings_merge_df['oxygentherapy']==1 ,'icustay_id'])
    O2_3= list(ventsettings_merge_df.loc[day3 & ventsettings_merge_df['oxygentherapy']==1 ,'icustay_id'])
    None_1= list(ventsettings_merge_df[(day1) & (ventsettings_merge_df['mechvent']==0) & (ventsettings_merge_df['oxygentherapy']==0) ]['icustay_id'])
    None_2= list(ventsettings_merge_df[(day2) & (ventsettings_merge_df['mechvent']==0) & (ventsettings_merge_df['oxygentherapy']==0) ]['icustay_id'])
    None_3= list(ventsettings_merge_df[(day3) & (ventsettings_merge_df['mechvent']==0) & (ventsettings_merge_df['oxygentherapy']==0) ]['icustay_id'])

    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(mech_1),'vent_day1']='Mech'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(mech_2),'vent_day2']='Mech'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(mech_3),'vent_day3']='Mech'

    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(O2_1),'vent_day1']='Oxygen'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(O2_2),'vent_day2']='Oxygen'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(O2_3),'vent_day3']='Oxygen'

    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(None_1),'vent_day1']='None'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(None_2),'vent_day2']='None'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(None_3),'vent_day3']='None'

    return(ventsettings_df_3dayannotation)

In [ ]:
#the following code is an alternative method to vent_categorization fxn written above. the output gives the # of vent events in a given timeframe
#for each icustay_id
def vent_count(final_pt_df2,ventsettings_df, time_var='t_0'):
    """
    overview: quantifies the # of ventilation events related to mechanical ventilation and oxygen therapy 
    for each 24 hr window after time_var (default= first ab dose )
       categories: 
        - Vent- had mech vent within 24hr period
        - Oxygen- had only oxygentherapy within 24hr period
        
    input: final cohort dataframe (by icustay_id/hadm_id) annotated with t_0, ventsettings_df
    output: a dataframe with every icustay_id as a row, and a count for each of the first 3 days based on ventilation recieved:
    """
    ventsettings_merge_df= pd.merge(ventsettings_df,
                                    final_pt_df2[['icustay_id',time_var]],
                                    left_on='icustay_id',
                                    right_on= 'icustay_id',
                                    how='left')

    day1= (
        (pd.to_datetime(ventsettings_merge_df[time_var])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('00:00:00','23:59:59'))
    day2= (
        (pd.to_datetime(ventsettings_merge_df[time_var])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('24:00:00','47:59:59'))
    day3= (
        (pd.to_datetime(ventsettings_merge_df[time_var])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('48:00:00','71:59:59'))


    #0-1
    icu_O2count1= (
        ventsettings_merge_df.loc[day1 & ventsettings_merge_df['oxygentherapy']==1 ,:]
                  .groupby('icustay_id')['oxygentherapy']
                  .value_counts())
    icu_mechcount1= (
        ventsettings_merge_df.loc[day1 & ventsettings_merge_df['mechvent']==1 ,:]
                .groupby('icustay_id')['mechvent']
                .value_counts())

    #1-2
    icu_O2count2= (
        ventsettings_merge_df.loc[day2 & ventsettings_merge_df['oxygentherapy']==1 ,:]
                  .groupby('icustay_id')['oxygentherapy']
                  .value_counts())

    icu_mechcount2= (
        ventsettings_merge_df.loc[day2 & ventsettings_merge_df['mechvent']==1 ,:]
                .groupby('icustay_id')['mechvent']
                .value_counts())

    #2-3
    icu_O2count3= (
        ventsettings_merge_df.loc[day3 & ventsettings_merge_df['oxygentherapy']==1 ,:]
                  .groupby('icustay_id')['oxygentherapy']
                  .value_counts())

    icu_mechcount3= (
        ventsettings_merge_df.loc[day3 & ventsettings_merge_df['mechvent']==1 ,:]
                .groupby('icustay_id')['mechvent']
                .value_counts())

    #0-1
    icu_O2count_df1= pd.DataFrame(icu_O2count1)
    icu_O2count_df1.columns = ['O2_count1']
    icu_O2count_df1= icu_O2count_df1.reset_index()

    icu_mechcount_df1= pd.DataFrame(icu_mechcount1)
    icu_mechcount_df1.columns = ['mech_count1']
    icu_mechcount_df1= icu_mechcount_df1.reset_index()

    #1-2
    icu_O2count_df2= pd.DataFrame(icu_O2count2)
    icu_O2count_df2.columns = ['O2_count2']
    icu_O2count_df2= icu_O2count_df2.reset_index()

    icu_mechcount_df2= pd.DataFrame(icu_mechcount2)
    icu_mechcount_df2.columns = ['mech_count2']
    icu_mechcount_df2= icu_mechcount_df2.reset_index()

    #2-3
    icu_O2count_df3= pd.DataFrame(icu_O2count3)
    icu_O2count_df3.columns = ['O2_count_3']
    icu_O2count_df3= icu_O2count_df3.reset_index()

    icu_mechcount_df3= pd.DataFrame(icu_mechcount3)
    icu_mechcount_df3.columns = ['mech_count_3']
    icu_mechcount_df3= icu_mechcount_df3.reset_index()

    #merging all together
    icu_vent_status= pd.merge(icu_O2count_df1,icu_O2count_df2, how='outer')
    icu_vent_status= pd.merge(icu_vent_status,icu_O2count_df3,  how='outer')
    icu_vent_status= pd.merge(icu_vent_status,icu_mechcount_df1,  how='outer')
    icu_vent_status= pd.merge(icu_vent_status,icu_mechcount_df2,  how='outer')
    icu_vent_status= pd.merge(icu_vent_status,icu_mechcount_df3,  how='outer')
    
    return(icu_vent_status.drop(['oxygentherapy','mechvent'], axis=1))

In [ ]:
ventcategory_df = vent_categorization(final_pt_df2, ventsettings_df, time_var='first_pos_else_neg_ssc' )
ventcategory_df['uom']='category'
ventcount_df = vent_count(final_pt_df2,ventsettings_df, time_var='first_pos_else_neg_ssc')

In [ ]:
ventcategory_df.head()

# daily SOFA score

# NEED TO RE-ADJUST FOR first_pos_else_neg_ssc

In [ ]:
##FACK, need to change it up to accomidate time windows. need sofascore for 3 days following first culture in ICU

### need to find sofa_pan, or daily sofa. 

In [ ]:
###requirements for sofa_pan
sofa <- scorecalc <- scorecomp <- vaso_cv    <-icu_18
                                             <-wt      <-icu_18
                                             <-echo2
                                          
                               <- vaso_mv    <-icu_18
                               <- pafi2      <-bloodgas_pan_aterial <- bloodgas_pan  <-icu_18
                                             <-ventelurations
                               <- vitals_pan <-icu_18
                               <- labs_pan   <-icu_18
                               <- uo_pan     <-icu_18
                               <- gcs_pan    <-icu_18

In [45]:
sofa_pan_df= clinvar_fxn(
    'sofa_pan',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/sofa_pan.sql' 
)


ProgrammingError: relation "scorecalc" does not exist
LINE 21: inner join scorecalc s
                    ^


time: 52.2 ms


In [50]:
#Yihangpan wrote a sql script and materialized view "sofa_pan" which gives the sofa score for each day in icu for each patient. 
#I now need to use the day # to find the associated t_0 to t_0+72 for each patient. 
sofa_df= sql_to_df_icu(sql_exe_show('select * from sofa_pan'))
sofa_df['icustay_id'].nunique() #15182/15207, pretty close. n=25 icustay id's don't have a sofa score? 

ProgrammingError: relation "sofa_pan" does not exist
LINE 1: select * from sofa_pan
                      ^


time: 29.7 ms


In [48]:
#adding in t_0 & icuadmit date
time_var= 'first_pos_else_neg_ssc' #'t_0'

sofa_df_merged= pd.merge(sofa_df,
                         final_pt_df2[['icustay_id',time_var,'ICU_admit']],
                         left_on= 'icustay_id',
                         right_on = 'icustay_id',
                         how='left') #n=240317

#sofa_df_merged['admit_t0_rounded'] = pd.to_datetime(sofa_df_merged['ICU_admit']).dt.round('1440min')

sofa_df_merged['day'] = sofa_df_merged['day']-1
sofa_df_merged['approx_charttime']=pd.to_timedelta((sofa_df_merged['day'])*24, unit='h') + pd.to_datetime(sofa_df_merged['ICU_admit'])

#rounding down the charttime to the day, so hours and minutes are ignored. 
sofa_df_merged['floor_charttime'] = sofa_df_merged['approx_charttime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, 24*(dt.hour//24))) 

NameError: name 'sofa_df' is not defined

time: 15.4 ms


In [216]:
sofa_df_72= sofa_df_merged.loc[
    (sofa_df_merged['floor_charttime'].between(
        pd.to_datetime(sofa_df_merged[time_var]),
        (pd.to_datetime(sofa_df_merged[time_var])+ pd.to_timedelta(2, unit='d'))
    )),:] 

#sofa_df_72= sofa_df_72.drop(54592) #dropping a duplicate row. 294198 had the only instance of 4 sofa score rows, one of which was a duplicate and deleted here.
sofa_df_72['icustay_id'].value_counts().describe() 

#n= 31930 ->37937 going from approx to floor charttime. ie this collapses icu_admit to remove all hour/minute and just display the day, similar to my previous steps. 

#this dataframe should be approximately all sofa scores for patients between t_0 and t_0+72, plus likely more. 

# the challenge was that I had to relate day# in sofa_pan to my t_0 date. the day # was based on the days after icu admission, where day1 = the first day (0 to 24 hours post admission). this was changed so day 0= 0 to 24 hours.
#To do this, I added day# (where day 0 is the first day) to icu admission date. 
#I then filtered on only the rows where this icuadmin + day# was between t_0 and t_0 + 72 hours. 

#since t_0 has only day resolution, and for that I ignored hours and only took the date (rounded down all hours/minutes/seconds). this is similar to how i made the t_0 date. 
#the problem this creates is that it widens the potential time window, so it theoretically can contain up to 95.99 hours, since hours on day 1 were collapsed to 0. 


time: 126 ms


# saving as csv

In [51]:
#useful for 
date= '25012019'

pd.DataFrame(ventcategory_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcategory_df.csv' %(date))
pd.DataFrame(ventcount_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcount_df.csv' %(date))

time: 111 ms


In [25]:
date= '27082018'

pd.DataFrame(vaso_dose_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vaso_dose_df.csv' %(date))
pd.DataFrame(ventsettings_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventsettings_df.csv' %(date))
pd.DataFrame(ventcategory_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcategory_df.csv' %(date))
pd.DataFrame(ventcount_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcount_df.csv' %(date))

pd.DataFrame(echodata_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_echodata_df.csv' %(date))
pd.DataFrame(weightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_weightfirstday_df.csv' %(date))
pd.DataFrame(heightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_heightfirstday_df.csv' %(date))
pd.DataFrame(labs_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_labs_all_nosummary_df.csv' %(date))

pd.DataFrame(vitals_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_df.csv' %(date))
pd.DataFrame(vitals_all_nosummary_nonull).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_nonull.csv' %(date)) #this is the more important one

# pd.DataFrame(gcsall_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcsall_df.csv' %(date))
pd.DataFrame(gcs72_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcs72_df.csv' %(date)) #saving 72hr window here


pd.DataFrame(rrt_all_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_rrt_all_df.csv' %(date))
pd.DataFrame(urine_output_all_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_urine_output_all_df.csv' %(date))
pd.DataFrame(uti_all_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_uti_all_df.csv' %(date))
pd.DataFrame(bg_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_bg_all_nosummary_df.csv' %(date))
pd.DataFrame(sofa_df_merged).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_sofa_df_merged.csv' %(date))

time: 2min 47s


In [34]:
# pd.DataFrame(bg_all_nosummary_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_bg_all_nosummary_df.csv' %(date))

time: 16 s


In [67]:
# pd.DataFrame(vaso_dose_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vaso_dose_df.csv' %(date))

time: 927 ms


# 72-hour time window filtering 8/28/18 
saving a new version of each clincal variable dataframe that is filtered to only 72 hour window after t_0 for each icustay_id 

- need to organize this better 

### reading in data if needed

In [256]:
# #final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/16082018_final_pt_df2.csv', index_col=0)

# #large import of all data 

# date= '27082018'

# vaso_dose_df =pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vaso_dose_df.csv' %(date), index_col=0)
# ventsettings_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventsettings_df.csv' %(date), index_col=0)
# ventcategory_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcategory_df.csv' %(date), index_col=0)
# ventcount_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcount_df.csv' %(date), index_col=0)

# echodata_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_echodata_df.csv' %(date), index_col=0)
# weightfirstday_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_weightfirstday_df.csv' %(date), index_col=0)
# heightfirstday_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_heightfirstday_df.csv' %(date), index_col=0)
# labs_all_nosummary_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_labs_all_nosummary_df.csv' %(date), index_col=0)
# vitals_all_nosummary_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_df.csv' %(date), index_col=0)
# gcsall_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcsall_df.csv' %(date), index_col=0)
# rrt_all_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_rrt_all_df.csv' %(date), index_col=0)
# urine_output_all_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_urine_output_all_df.csv' %(date), index_col=0)
# uti_all_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_uti_all_df.csv' %(date), index_col=0)
# bg_all_nosummary_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_bg_all_nosummary_df.csv' %(date), index_col=0)


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 32.6 s


In [9]:
def hour_72_window(df, final_pt_df2,timevar):
    try:
        df= pd.merge(df, final_pt_df2[['icustay_id','t_0']], left_on= 'icustay_id', right_on = 'icustay_id') #n=240317
        df['delta']= pd.to_datetime(df[timevar]) - pd.to_datetime(df['t_0'])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=0),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=3),:]
    except KeyError or NameError:
        df= pd.merge(df, final_pt_df2[['hadm_id','t_0']], left_on= 'hadm_id', right_on = 'hadm_id') #n=240317
        df['delta']= pd.to_datetime(df[timevar]) - pd.to_datetime(df['t_0'])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=0),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=3),:]
    return(df_after_t0)

time: 12.9 ms


In [272]:
###list of data w/ 72 hour adjustments
#vaso_dose_df - vaso_dose_72 -
#ventsettings_df - ventsettings_72 -
#ventcategory_df - ventcategory_df - 
#ventcount_df - ventcount_df -
#echodata_df - echodata_72 -
#labs_all_nosummary_df- labs_all_nosummary_72 -
#vitals_all_nosummary_df - vitals_all_nosummary_72 -
#gcsall_df - Gcs72_df **modified the gcs in python above. within 72 hour window. -
#rrt_all_df - rrt_merged_allpt_df *modified the rrt in python above. within 72 hour window.  -
#uti_all_df - uti_all_72 -
#bg_all_nosummary_df - bg_all_nosummary_72
#sofa_df_72 #already within 72hour window

#urine_output_all_df - . xxx this also doesn't have times, but this won't be used much so i didn't bother editing.

vaso_dose_72= hour_72_window(vaso_dose_df, final_pt_df2, 'starttime')
ventsettings_72= hour_72_window(ventsettings_df, final_pt_df2, "charttime")
echodata_72= hour_72_window(echodata_df, final_pt_df2, 'charttime')
labs_all_nosummary_72= hour_72_window(labs_all_nosummary_df, final_pt_df2, "charttime")
vitals_all_nosummary_72 = hour_72_window(vitals_all_nosummary_df, final_pt_df2, "charttime")
uti_all_72 = hour_72_window(uti_all_df, final_pt_df2, 'charttime')
bg_all_nosummary_72 = hour_72_window(bg_all_nosummary_df, final_pt_df2, 'charttime')

time: 36.7 s


In [37]:
bg_all_nosummary_72 = hour_72_window(bg_all_nosummary_df, final_pt_df2, 'charttime')
vitals_all_nosummary_72 = hour_72_window(vitals_all_nosummary_df, final_pt_df2, "charttime")

time: 33.3 s


# filtering to patients with bare minimum vital numbers
06.1-QC_and_missingness found that 3% or so of patients don't have baseline vitals counts. this is filtering the patients to only those who have this baseline value.

as of 10/12/18, this code has not been implemented in here and is performed after importing. 

In [21]:
def df_subset(df):
    try:
        df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id),:]
    except KeyError or NameError:
        try:
            df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id),:]
        except KeyError or NameError:
            df = df.loc[df.loc[:,'subject_id'].isin(patients),:]
    return(df)

time: 5.12 ms


In [40]:
# subsetting each dataframe to only patients in final_patients_df2_v:
dataframe_list= [
    ventcategory_df, vaso_dose_72, echodata_72,
    labs_all_nosummary_72, weightfirstday_df,
    heightfirstday_df, vitals_all_nosummary_72,
    uti_all_72, bg_all_nosummary_72,
    rrt_merged_allpt_df, gcs72_df, sofa_df_72
]

(
ventcategory_df, vaso_dose_72, echodata_72,
labs_all_nosummary_72, weightfirstday_df,
heightfirstday_df, vitals_all_nosummary_72,
uti_all_72, bg_all_nosummary_72,
rrt_merged_allpt_df, gcs72_df, sofa_df_72
) = (df_subset(df) for df in dataframe_list)

NameError: name 'ventcategory_df' is not defined

time: 57.4 ms


In [274]:
date= '27082018'

pd.DataFrame(vaso_dose_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date))
pd.DataFrame(ventsettings_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_ventsettings_72.csv' %(date))

#vent category and count are already 72hour
pd.DataFrame(ventsettings_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventsettings_df.csv' %(date))
pd.DataFrame(ventcategory_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcategory_df.csv' %(date))

pd.DataFrame(echodata_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date))
pd.DataFrame(labs_all_nosummary_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date))

#vent category and count are already limited to first day
pd.DataFrame(weightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date))
pd.DataFrame(heightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date))

pd.DataFrame(vitals_all_nosummary_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date))
pd.DataFrame(uti_all_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date))
pd.DataFrame(bg_all_nosummary_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date))

pd.DataFrame(rrt_merged_allpt_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date))
pd.DataFrame(gcs72_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date))

pd.DataFrame(sofa_df_72).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_sofa_df_72.csv' %(date))


NameError: name 'Gcs72_df' is not defined

time: 2min
